# Machine Learning Approach single model approach

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import statsmodels.formula.api as smf 
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split 
import itertools
%matplotlib inline
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_squared_error #MSE
from sklearn.metrics import make_scorer, r2_score
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

## Modeling with Statistically significant variables

In [6]:
df_ml = pd.read_csv('/Users/daisukeyagyu/Desktop/UChicago/Courses/Capstone/00_Implementation/Delete_AfterProject/01_Outputs/modeling_1st/df_model_1st.csv').dropna()

In [4]:
target = 'TARGET_Weighted_RetailPrice'

In [5]:
X = df_ml.drop(target, axis=1)

In [6]:
y = df_ml[[target]]

In [7]:
y = np.log(y)

In [8]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [13]:
def rmse(y_true,y_pred):
    rmse = np.sqrt(mean_squared_error(y_true,y_pred))
    print('rmse',rmse)
    return rmse

In [24]:
def rsquare(y_true,y_pred):
    rsquare = r2_score(y_true, y_pred)
    print('rsquare',rsquare)
    return rsquare

In [163]:
def randomsearchRegression(model, parameters):
    from sklearn.model_selection import RandomizedSearchCV
    searcher = RandomizedSearchCV(model, parameters, cv=10, scoring=make_scorer(rsquare,greater_is_better=True))
    searcher.fit(train_X, train_y)
    print("Best CV params", searcher.best_params_)
    print("Best CV Rsquare", -searcher.best_score_)
    print("Best train Rsquare", -searcher.score(train_X, train_y))
    print("Test RMSE of best random search hypers:", -searcher.score(test_X, test_y))
    return searcher.best_params_,  -searcher.best_score_, -searcher.score(test_X, test_y), -searcher.score(train_X, train_y)

In [26]:
models_dict = {}
models_dict['random_forest'] = RandomForestRegressor()
models_dict['adaboost'] = AdaBoostRegressor()
models_dict['xgboost'] = xgb.XGBRegressor()

In [27]:
result_dict = {}
parameters_dct = {}
parameters_dct['random_forest'] = {"n_estimators": [10, 100, 300, 500,1000],
                                    "max_depth": [i for i in range(3,11)],
                                    'max_features' : [i for i in range(1,train_X.shape[1])],
                                    "random_state": [1],
                                    "min_samples_leaf": [i for i in range(1, 21)]}
parameters_dct['adaboost'] = {"n_estimators": [10, 100, 300, 500,1000],
                                    'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1],
                                    'random_state': [1]}
parameters_dct['xgboost'] = {"n_estimators": [10, 100, 300, 500,1000],
                                    'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1],
                                    'max_depth':[1,3,5,7,10],
                                    'gamma': [0.01*i for i in range(0,1000,100)],
                                    'random_state': [1]}


In [28]:
model_list = ['random_forest', 'adaboost', 'xgboost']

In [29]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9538096844383922
rsquare 0.9546249544781541
rsquare 0.9422476392878749
rsquare 0.9442357610875953
rsquare 0.9434946531735251
rsquare 0.9512303378504705
rsquare 0.9477854791483138
rsquare 0.949214407093239
rsquare 0.9471651419109642
rsquare 0.9526915211353923
rsquare 0.9675915574917777
rsquare 0.9676334907224567
rsquare 0.9595392200591045
rsquare 0.962017915370204
rsquare 0.9590629863002448
rsquare 0.9607998671941922
rsquare 0.9623153379536694
rsquare 0.9629505718625285
rsquare 0.9620728016886918
rsquare 0.9610362619218387
rsquare 0.9191966360838282
rsquare 0.923883631496405
rsquare 0.9087436446569143
rsquare 0.909819637633615
rsquare 0.9181770057139893
rsquare 0.9156308831241492
rsquare 0.9164998361615151
rsquare 0.9270946179392808
rsquare 0.9140667627071907
rsquare 0.9109493394901834
rsquare 0.910242877712448
rsquare 0.9043597448280768
rsquare 0.8985683558750398
rsquare 0.9069928484213383
rsquare 0.9098295274005292
rsquare 0.9007178800841483
rsquare 0.909797404

### Random Forest result

In [9]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 300, min_samples_leaf= 18, max_features= 391, max_depth= 9)

In [10]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=9, max_features=391, min_samples_leaf=18,
                      n_estimators=300, random_state=1)

In [11]:
pred_y = best_randomforest.predict(test_X)

In [14]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 4049.700363216011


4049.700363216011

In [15]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9500053336086076

In [16]:
RF_importances = best_randomforest.feature_importances_

In [17]:
feature_names = list(X.columns)

In [18]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [19]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_maximum_torque_Nm,0.257360
SPEC_maximum_torque_lb_ft,0.252914
SPEC_driver_selectable,0.118184
SPEC_wheelrim_width_inch,0.095657
SPEC_published_kerb_weight_kg,0.055459
...,...
SPEC_Active_brakelights,0.000000
SPEC_removable,0.000000
SPEC_Night_vision,0.000000
Segment_COMPACT_SUV,0.000000


### Adaboost result

In [20]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=300, learning_rate=1)


In [21]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=300, random_state=1)

In [22]:
pred_y = best_adaboost.predict(test_X)

In [23]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 5308.800693319489


5308.800693319489

In [24]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9162368780276362

In [25]:
AB_importances = best_adaboost.feature_importances_

In [26]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [27]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_wheelrim_width_inch,0.395762
SPEC_driver_selectable,0.103517
SPEC_maximum_torque_Nm,0.079806
SPEC_maximum_torque_lb_ft,0.044832
SPEC_published_kerb_weight_kg,0.037675
...,...
SPEC_Compass,0.000000
SPEC_Warranty_full_service,0.000000
SPEC_rear_control_for_climate_control,0.000000
Segment_COMPACT_SUV,0.000000


### XGboost result

In [28]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=500,max_depth=5, learning_rate=0.1, gamma=3)

In [29]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=3, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [30]:
pred_y = best_xgboost.predict(test_X)

In [31]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 5638.858853643156


5638.858853643156

In [32]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8846921956832713

In [33]:
XGB_importances = best_xgboost.feature_importances_

In [34]:
XGB_importances = pd.DataFrame(XGB_importances, columns=['importances'], index=feature_names)

In [35]:
XGB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_FRPGseat_reclining_manual,0.248584
SPEC_driver_selectable,0.110374
SPEC_maximum_torque_Nm,0.110021
SPEC_maximum_torque_lb_ft,0.054282
SPEC_wheelrim_width_inch,0.050515
...,...
SPEC_FRDRseat_thighsupport_memory,0.000000
PROFILE_Make_KIA,0.000000
SPEC_Floor_mats,0.000000
SPEC_Driver_modes,0.000000


In [36]:
RF_vars = list(RF_importances.sort_values(by='importances', ascending=False)[:200].index)

In [37]:
AB_vars = list(AB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [38]:
XGB_vars = list(XGB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [39]:
intercection1 = set(RF_vars).intersection(set(AB_vars))
intercection2 = set(AB_vars).intersection(set(XGB_vars))
intercection3 = set(RF_vars).intersection(set(XGB_vars))
featureimp_variables = list(intercection1.union(intercection2).union(intercection3))

In [197]:
pd.DataFrame(featureimp_variables).to_csv('ModelwReducedVariables_union_of_intersections.csv', index=False)

## Model with reduced variables Union of Intersections (190 variables)

In [164]:
X = df_ml.drop(target, axis=1)

In [169]:
X_featureimp = X[featureimp_variables]

In [167]:
y = df_ml[[target]]

In [168]:
y = np.log(y)

In [170]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [171]:
result_dict_2 = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9675836060344343
rsquare 0.9668438079176254
rsquare 0.9579843841339822
rsquare 0.961159334683553
rsquare 0.9571148677716259
rsquare 0.9585587699803452
rsquare 0.9604206026133372
rsquare 0.9628550120693222
rsquare 0.9603525651448508
rsquare 0.9597715945667755
rsquare 0.9569906867637625
rsquare 0.9599274988384805
rsquare 0.9489190106060107
rsquare 0.9516670054991283
rsquare 0.9515521702389302
rsquare 0.9513087273514136
rsquare 0.955048134795758
rsquare 0.9572031747252833
rsquare 0.9495472077369748
rsquare 0.9521114960736599
rsquare 0.9693692758362635
rsquare 0.9678784043592824
rsquare 0.9587348338692012
rsquare 0.9639418473823154
rsquare 0.9582453026531011
rsquare 0.9608826115988957
rsquare 0.9620856124502962
rsquare 0.9644758250584787
rsquare 0.9629157096362683
rsquare 0.9616906283778102
rsquare 0.8160530397410896
rsquare 0.8234381592840058
rsquare 0.7849917560847408
rsquare 0.7791203486398062
rsquare 0.7722861345479098
rsquare 0.7986029874898524
rsquare 0.783502

### Random Forest result

In [172]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 300, min_samples_leaf= 13, max_features= 246, max_depth= 10)

In [173]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=10, max_features=246, min_samples_leaf=13,
                      n_estimators=300, random_state=1)

In [174]:
pred_y = best_randomforest.predict(test_X)

In [175]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3881.441889213967


3881.441889213967

In [176]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9558539735114822

In [177]:
RF_importances = best_randomforest.feature_importances_

In [181]:
feature_names = list(X_featureimp.columns)

In [182]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [183]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_maximum_torque_Nm,0.270395
SPEC_maximum_torque_lb_ft,0.234665
SPEC_driver_selectable,0.154326
SPEC_doormirror_autodimming,0.075720
SPEC_wheelrim_width_inch,0.071469
...,...
SPEC_FRPGseat_RRPGcontroll,0.000000
SPEC_Online_entertainment,0.000000
SPEC_wheelrim_composite,0.000000
SPEC_Head_restraints,0.000000


### Adaboost result

In [184]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=1000, learning_rate=1)


In [185]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=1000, random_state=1)

In [186]:
pred_y = best_adaboost.predict(test_X)

In [187]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 5257.9365174909635


5257.9365174909635

In [188]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9175931273810543

In [189]:
AB_importances = best_adaboost.feature_importances_

In [190]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [191]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_wheelrim_width_inch,0.409691
SPEC_driver_selectable,0.100642
SPEC_maximum_torque_Nm,0.066915
SPEC_maximum_torque_lb_ft,0.041085
SPEC_published_kerb_weight_kg,0.040357
...,...
SPEC_multi_touch_screen,0.000000
SPEC_FRDRseat_lumbar_2way,0.000000
SPEC_Compressor_turbo,0.000000
SPEC_RRseatbelt_DRPG,0.000000


### XGboost result

In [192]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=500, max_depth=7, learning_rate=0.1, gamma=0)

In [193]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [194]:
pred_y = best_xgboost.predict(test_X)

In [195]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1949.5178645579676


1949.5178645579676

In [196]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9895369430828271

## Model with reduced variables of Intersections (85 variables)

In [40]:
featureimp_variables = list(set(RF_vars).intersection(set(XGB_vars)).intersection(set(AB_vars)))

In [41]:
pd.DataFrame(featureimp_variables).to_csv('ModelwReducedVariables_intersection.csv', index=False)

In [199]:
X = df_ml.drop(target, axis=1)

In [200]:
X_featureimp = X[featureimp_variables]

In [201]:
y = df_ml[[target]]

In [202]:
y = np.log(y)

In [203]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [204]:
result_dict_3 = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9643440623177465
rsquare 0.9636528381079073
rsquare 0.9556722613650199
rsquare 0.9595636527326475
rsquare 0.9529285706081869
rsquare 0.9576295634806629
rsquare 0.9564400782744479
rsquare 0.9593120170727688
rsquare 0.9589724127665777
rsquare 0.9567304693452421
rsquare 0.8729323052263909
rsquare 0.875461298511393
rsquare 0.8501031267962955
rsquare 0.842748468107417
rsquare 0.8424260686760724
rsquare 0.8614667303525356
rsquare 0.848336950633878
rsquare 0.8735104662257593
rsquare 0.8568720840076531
rsquare 0.8529127941056909
rsquare 0.9120760365271507
rsquare 0.9145694079841483
rsquare 0.8968144251791184
rsquare 0.8922972573045204
rsquare 0.8943625948510323
rsquare 0.9028564611554668
rsquare 0.8940844451763851
rsquare 0.9082686616705202
rsquare 0.8994620003875687
rsquare 0.9012124357804931
rsquare 0.9613435281770053
rsquare 0.9615081522400349
rsquare 0.9530669201448637
rsquare 0.952470446444797
rsquare 0.9530330091202993
rsquare 0.951772977014769
rsquare 0.953579343

### Random Forest result

In [205]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 1000, min_samples_leaf= 15, max_features= 490, max_depth= 10)

In [206]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=10, max_features=490, min_samples_leaf=15,
                      n_estimators=1000, random_state=1)

In [208]:
pred_y = best_randomforest.predict(test_X)

In [209]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 4079.286629739909


4079.286629739909

In [210]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9505865074443626

In [211]:
RF_importances = best_randomforest.feature_importances_

In [212]:
feature_names = list(X_featureimp.columns)

In [213]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [214]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_maximum_torque_Nm,0.261076
SPEC_maximum_torque_lb_ft,0.247055
SPEC_driver_selectable,0.149659
SPEC_wheelrim_width_inch,0.078755
SPEC_doormirror_autodimming,0.074940
...,...
SPEC_multi_touch_screen,0.000140
SPEC_Crash_test_results,0.000120
SPEC_Seat_belt_warning,0.000118
SPEC_SideAirbag_headprotect,0.000112


### Adaboost result

In [215]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=500, learning_rate=1)


In [216]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=500, random_state=1)

In [217]:
pred_y = best_adaboost.predict(test_X)

In [218]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 5517.975813032049


5517.975813032049

In [219]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9078445196499934

In [220]:
AB_importances = best_adaboost.feature_importances_

In [221]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [222]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_wheelrim_width_inch,0.392013
SPEC_driver_selectable,0.110408
SPEC_maximum_torque_Nm,0.092046
SPEC_maximum_torque_lb_ft,0.054897
SPEC_published_kerb_weight_kg,0.040613
...,...
SPEC_manual_mode,0.000000
PROFILE_Make_MERCEDES,0.000000
SPEC_SideAirbag_headprotect,0.000000
SPEC_multi_touch_screen,0.000000


### XGboost result

In [223]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=100, max_depth=3, learning_rate=1, gamma=0)

In [224]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [225]:
pred_y = best_xgboost.predict(test_X)

In [226]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3205.416102135327


3205.416102135327

In [227]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9716159826252065

# Non Linear modeling

In [ ]:
df_ml

In [42]:
intercection1 = set(RF_vars).intersection(set(AB_vars))
intercection2 = set(AB_vars).intersection(set(XGB_vars))
intercection3 = set(RF_vars).intersection(set(XGB_vars))
featureimp_variables = list(intercection1.union(intercection2).union(intercection3))

In [43]:
df_ml[featureimp_variables+['TARGET_Weighted_RetailPrice']]

,SPEC_SideAirbag_headprotect,SPEC_headlight_Halogen,SPEC_luxurytrim_instrumentpanel_gloss,SPEC_CountryOfAssembly_Spain,SPEC_FRPGseat_tilt_memory,SPEC_RRseatbelt_center3point,SPEC_transmission_AT,SPEC_Airconditioningtype_manual,SPEC_FRPGseat_height_electric,SPEC_step_rear,...,SPEC_Airconditioningtype_auto,SPEC_fueltype_E85,SPEC_Automatic_door_closing,SPEC_amount,SPEC_Anti_theft_protection,SPEC_reconfigurable,SPEC_RRseatbelt_DRPGCenter,SPEC_Luxury_inner_roof_upholstery,SPEC_Cornering_brake_control,TARGET_Weighted_RetailPrice
0,0,0,0,0,0,1,1,0,1,0,...,1,0,1,895.0,1,0,1,0,1,44290.000000
1,0,0,0,0,0,1,1,0,1,0,...,1,0,1,895.0,1,0,1,0,1,42290.000000
2,0,0,0,0,0,1,1,0,1,0,...,1,0,1,895.0,1,0,1,0,1,56505.000000
3,0,0,0,0,0,1,1,0,1,0,...,1,0,1,895.0,1,0,1,0,1,50565.000000
4,0,0,0,0,0,1,1,0,1,0,...,1,0,1,895.0,1,0,1,0,1,54505.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13346,0,0,0,0,1,1,1,0,0,0,...,1,0,1,1295.0,0,1,1,0,0,49988.150685
13347,0,0,0,0,1,1,1,0,0,0,...,1,0,1,1295.0,0,1,1,0,0,53839.169247
13348,0,0,0,0,1,0,1,0,0,0,...,1,0,1,1295.0,0,1,0,0,0,49945.000000
13349,0,0,0,0,1,0,1,0,0,0,...,1,0,1,1295.0,0,1,0,0,0,49988.150685


In [59]:
# 6. transform data
data = df_ml[featureimp_variables+['TARGET_Weighted_RetailPrice']]

# 7. set formula and family 
formula = "TARGET_Weighted_RetailPrice ~ 1 +" + ' + '.join(featureimp_variables) 
link = sm.genmod.families.links.log()
family = sm.families.Gamma(link = link)

# 8.Create GML model
model = smf.glm(formula=formula, data=data, family=family) 
# 9. Fit the model
result = model.fit() 
# 10. Get summary
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Generalized Linear Model Regression Results                      
=======================================================================================
Dep. Variable:     TARGET_Weighted_RetailPrice   No. Observations:                13351
Model:                                     GLM   Df Residuals:                    13165
Model Family:                            Gamma   Df Model:                          185
Link Function:                             log   Scale:                       0.0052656
Method:                                   IRLS   Log-Likelihood:            -1.2465e+05
Date:                         Tue, 04 Jul 2023   Deviance:                       67.621
Time:                                 01:23:35   Pearson chi2:                     69.3
No. Iterations:                             29   Pseudo R-squ. (CS):              1.000
Covariance Type:                     nonrobust                                         
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             3.7853      0.048     78.480      0.000       3.691       3.880
SPEC_SideAirbag_headprotect                          -0.0039      0.007     -0.566      0.572      -0.017       0.010
SPEC_headlight_Halogen                               -0.0088      0.003     -2.881      0.004      -0.015      -0.003
SPEC_luxurytrim_instrumentpanel_gloss                -0.0103      0.005     -1.906      0.057      -0.021       0.000
SPEC_CountryOfAssembly_Spain                          0.0691      0.009      7.697      0.000       0.052       0.087
SPEC_FRPGseat_tilt_memory                            -0.2172      0.020    -11.003      0.000      -0.256      -0.179
SPEC_RRseatbelt_center3point                          0.0713      0.010      7.015      0.000       0.051       0.091
SPEC_transmission_AT                                  1.8508      0.024     75.572      0.000       1.803       1.899
SPEC_Airconditioningtype_manual                       0.0402      0.008      5.240      0.000       0.025       0.055
SPEC_FRPGseat_height_electric                         0.0117      0.003      3.579      0.000       0.005       0.018
SPEC_step_rear                                       -0.0207      0.003     -6.133      0.000      -0.027      -0.014
SPEC_Underbody_protection                            -0.0043      0.002     -1.734      0.083      -0.009       0.001
SPEC_manual_mode                                      0.0016      0.002      0.682      0.495      -0.003       0.006
SPEC_Reading_lights                                  -0.0046      0.003     -1.334      0.182      -0.011       0.002
SPEC_Seatmaterial_leather                             0.0124      0.002      5.572      0.000       0.008       0.017
SPEC_VentilatedDiskBrake_num                          0.0008      0.001      0.655      0.512      -0.002       0.003
SPEC_RRseatbelt_DRPG                                  0.0811      0.005     17.847      0.000       0.072       0.090
NCBS_OVERALL_Recommendation_Likeliness                0.0185      0.003      6.164      0.000       0.013       0.024
SPEC_ManualTransmission                              -0.1127      0.011    -10.534      0.000      -0.134      -0.092
SPEC_FRPGseat_reclining_electric                     -0.2577      0.019    -13.497      0.000      -0.295      -0.220
SPEC_automatic_recirculation                          0.0391      0.004     11.051      0.000       0.032       0.046
Segment_FULLSIZE_PICKUP                              -0.0679      0.005    -13.789      0.000      -0.078      -0.058
NCBS_OVERALL_Experience           

In [60]:
df_pvalues = pd.DataFrame(result.pvalues,columns=['pvalues'])
significant_values = list(df_pvalues[df_pvalues['pvalues']<0.05].index)

In [61]:
significant_values.remove('Intercept')


In [62]:
len(significant_values)

150

In [70]:
selected_var = ['item','item2']
'+'.join(selected_var)

'item+item2'

In [78]:
significant_values_backup = significant_values.copy()

In [81]:
aic_dict = {}
selected_var = []
min_aic = 9999999999999999
for i in range(len(significant_values)):
    updated = 0
    bestaic_var = ''
    for item in significant_values:
        model_combi = selected_var + [item]

        formula = "TARGET_Weighted_RetailPrice ~ 1 +" + '+'.join(model_combi)  
        link = sm.genmod.families.links.log()
        family = sm.families.Gamma(link = link)
        model = smf.glm(formula=formula, data=data, family=family) 
        result = model.fit() 
        
        aic_dict[str('+'.join(model_combi))] = result.aic
        if result.aic < min_aic:
            min_aic = result.aic
            updated += 1
            bestaic_var = item

    if updated > 0:
        selected_var.append(bestaic_var)
        significant_values.remove(bestaic_var)
    else:
        break

    print(i, bestaic_var)

0 SPEC_maximum_torque_Nm
1 SPEC_FRPGseat_reclining_manual
2 PROFILE_Make_PORSCHE
3 SPEC_amount
4 SPEC_Airconditioningtype_auto
5 SPEC_fueltype_unleaded
6 SPEC_published_kerb_weight_kg
7 SPEC_driver_selectable
8 SPEC_automatic_recirculation
9 SPEC_4WD_fulltime
10 SPEC_Power_windows
11 SPEC_screen_size_in
12 Segment_FULLSIZE_PICKUP
13 Segment_SPORTS_PERFORMANCE
14 Segment_MID_LUXURY
15 SPEC_subwoofer
16 SPEC_Ceramic_brakes
17 SPEC_wheelrim_steel
18 PROFILE_Make_MERCEDES
19 Segment_ENTRY
20 SPEC_surround_sound
21 SPEC_Steering_suede
22 SPEC_auto_dimming
23 SPEC_Drivenwheels_4x4
24 NCBS_OVERALL_Noise_Vibration_Harshness
25 NCBS_OVERALL_Value_For_The_Money
26 SPEC_FRDRseat_reclining_manual
27 NCBS_OVERALL_Impression_of_Durability_Reliability
28 SPEC_Body_side_molding
29 SPEC_heated816
30 SPEC_Alternate_suspension
31 SPEC_Driver_modes
32 SPEC_Luxury_inner_roof_upholstery
33 SPEC_headrestraints_heightadjust
34 SPEC_Smart_card___smart_key
35 SPEC_Continentgroup_OTH
36 SPEC_Front_seat
37 SPEC_E

In [83]:
pd.DataFrame.from_dict(aic_dict, orient='index', columns=['aic']).sort_values(by='aic').to_csv('AIC_result.csv')

In [85]:
best_aic_model = pd.DataFrame.from_dict(aic_dict, orient='index', columns=['aic']).sort_values(by='aic').index[0]

In [86]:
target = 'TARGET_Weighted_RetailPrice'

In [87]:
X = df_ml.drop(target, axis=1)

In [88]:
y = df_ml[[target]]

In [89]:
y = np.log(y)

In [90]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [94]:
# 6. transform data
data = pd.concat([train_X[best_aic_model.split('+')], train_y], axis=1)

# 7. set formula and family 
formula = "TARGET_Weighted_RetailPrice ~ 1 +" + best_aic_model 
link = sm.genmod.families.links.log()
family = sm.families.Gamma(link = link)

# 8.Create GML model
model = smf.glm(formula=formula, data=data, family=family) 
# 9. Fit the model
result = model.fit() 
# 10. Get summary
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Generalized Linear Model Regression Results                      
=======================================================================================
Dep. Variable:     TARGET_Weighted_RetailPrice   No. Observations:                 9345
Model:                                     GLM   Df Residuals:                     9207
Model Family:                            Gamma   Df Model:                          137
Link Function:                             log   Scale:                      4.7383e-05
Method:                                   IRLS   Log-Likelihood:                 11320.
Date:                         Tue, 04 Jul 2023   Deviance:                      0.43574
Time:                                 14:23:44   Pearson chi2:                    0.436
No. Iterations:                             54   Pseudo R-squ. (CS):              1.000
Covariance Type:                     nonrobust                                         
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             1.6167      0.004    437.921      0.000       1.609       1.624
SPEC_maximum_torque_Nm                             4.783e-05   1.39e-06     34.354      0.000    4.51e-05    5.06e-05
SPEC_FRPGseat_reclining_manual                       -0.0304      0.002    -13.280      0.000      -0.035      -0.026
PROFILE_Make_PORSCHE                                  0.0387      0.001     25.950      0.000       0.036       0.042
SPEC_amount                                        2.509e-05   6.45e-07     38.887      0.000    2.38e-05    2.64e-05
SPEC_Airconditioningtype_auto                         0.0056      0.001      6.496      0.000       0.004       0.007
SPEC_fueltype_unleaded                               -0.0112      0.001    -19.521      0.000      -0.012      -0.010
SPEC_published_kerb_weight_kg                      1.951e-05   5.58e-07     34.988      0.000    1.84e-05    2.06e-05
SPEC_driver_selectable                                0.0047      0.000     12.584      0.000       0.004       0.005
SPEC_automatic_recirculation                          0.0043      0.000     11.764      0.000       0.004       0.005
SPEC_4WD_fulltime                                    -0.0026      0.000     -6.785      0.000      -0.003      -0.002
SPEC_Power_windows                                    0.0061      0.000     13.181      0.000       0.005       0.007
SPEC_screen_size_in                                  -0.0003   7.39e-05     -3.813      0.000      -0.000      -0.000
Segment_FULLSIZE_PICKUP                              -0.0058      0.000    -12.688      0.000      -0.007      -0.005
Segment_SPORTS_PERFORMANCE                            0.0243      0.001     30.169      0.000       0.023       0.026
Segment_MID_LUXURY                                    0.0124      0.001     21.702      0.000       0.011       0.014
SPEC_subwoofer                                        0.0019      0.000      7.008      0.000       0.001       0.002
SPEC_Ceramic_brakes                                   0.0302      0.002     18.949      0.000       0.027       0.033
SPEC_wheelrim_steel                                   0.5298      0.001    390.415      0.000       0.527       0.532
PROFILE_Make_MERCEDES                                 0.0138      0.001     20.520      0.000       0.013       0.015
Segment_ENTRY                                        -0.0123      0.001    -19.363      0.000      -0.014      -0.011
SPEC_surround_sound                                   0.0029      0.000      9.438      0.000       0.002       0.003
SPEC_Steering_suede               

In [98]:
pred_y = result.predict(test_X)

In [99]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 4569.268574782549


4569.268574782549

In [100]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9444881203056495